In [ ]:
import pandas as pd
import datetime
from pathlib import Path
from scipy.io import loadmat
from typing import Literal



In [ ]:
def load_data(path: Path | str, target_operation: Literal["charge", "discharge"]) -> pd.DataFrame:
    path = Path(path)
    mat = loadmat(path)  # load mat-file
    data = mat[path.stem]["cycle"][0, 0]

    parsed_data = []
    for cycle in data:
        for operation_id, operation in enumerate(cycle):
            operation_type = operation["type"][0]
            if operation_type != target_operation:
                continue

            for fields in operation["data"]:
                if operation_type == "charge":
                    operation_data = pd.DataFrame({
                        k: fields[k][0][0]
                        for k in fields.dtype.fields
                    })
                elif operation_type == "discharge":
                    operation_data = pd.DataFrame({
                        k: fields[k][0][0]
                        for k in fields.dtype.fields
                        if k != "Capacity"
                    })
                    operation_data["Capacity"] = fields["Capacity"][0][0, 0]      

            operation_data["operation_id"] = operation_id
            operation_data["temperature"] = operation["ambient_temperature"][0, 0]
            operation_data["type"] = operation_type
            operation_data["start_time"] = datetime.datetime(*(int(t) for t in operation["time"][0]))

            parsed_data.append(operation_data)
    
    return pd.concat(parsed_data, ignore_index=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
paths = sorted(Path("5. Battery Data Set/3. BatteryAgingARC_25-44").glob("*.mat"))

for p in paths:
    data = load_data(p, "discharge")
    sns.scatterplot(data=data, x="start_time", y="Capacity")
    plt.title(Path(p).stem)
    plt.show()